Pré-processamento e filtragem de dados

In [30]:
# Importar Bibliotecas Necessárias
import os
import pandas as pd

# Definir a função para pré-processamento de dados
def preprocess_data(raw_data_file):
    """
    Função para pré-processamento e filtragem de dados.
    
    Args:
    - raw_data_file (str): Caminho do arquivo CSV contendo os dados brutos.
    
    Returns:
    - processed_data (DataFrame): DataFrame pré-processado e filtrado.
    """
    # Carregar os dados brutos do arquivo CSV
    df_raw = pd.read_csv(raw_data_file, sep=',')
    
    # Filtrar os dados onde o valor de shot_type é igual a '2PT Field Goal'
    df_filtered = df_raw[df_raw['shot_type'] == '2PT Field Goal']

    # Filtrar o DataFrame para incluir apenas as colunas especificadas
    columns_to_keep = ['lat', 'lon', 'minutes_remaining', 'period', 'playoffs', 'shot_distance', 'shot_made_flag']
    processed_data = df_filtered[columns_to_keep].copy()

    # Tratar dados ausentes, se houver
    processed_data.dropna(inplace=True)

    # Substituir valores faltantes na coluna 'shot_made_flag' por 0 (indicando que o arremesso foi errado)
    processed_data['shot_made_flag'] = processed_data['shot_made_flag'].astype(int)

    return processed_data

# Caminho do arquivo CSV contendo os dados brutos
raw_data_file = 'Proj_Kobe-Bryant-Shot-Selection/data/raw/data.csv'

# Executar a função de pré-processamento de dados
df_kobe = preprocess_data(raw_data_file)

# Exibir as primeiras linhas do DataFrame pré-processado
print("Dimensão do DataFrame filtrado e pré-processado:", df_kobe.shape)
df_kobe.head()

# Salvar o DataFrame pré-processado em arquivos Parquet de desenvolvimento e produção
output_file_dev = 'Proj_Kobe-Bryant-Shot-Selection/data/processed/dataset_kobe_dev.parquet'
output_file_prod = 'Proj_Kobe-Bryant-Shot-Selection/data/processed/dataset_kobe_prod.parquet'
df_kobe.to_parquet(output_file_dev)
df_kobe.to_parquet(output_file_prod)

print("Arquivos Parquet de dados brutos de prod e dev criados com sucesso.")

Dimensão do DataFrame filtrado e pré-processado: (20285, 7)
Arquivos Parquet de dados brutos de prod e dev criados com sucesso.


Divisão dos Dados em Conjuntos de Treino (80%) e Teste (20 %)

In [36]:
def split_data(data, test_size=0.2):

    X = data.drop('shot_made_flag', axis=1)
    y = data['shot_made_flag']
    
    # Dividir os dados de forma aleatória e estratificada
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
    
    return X_train, X_test, y_train, y_test

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = split_data(df_kobe)

# Exibir informações sobre a divisão
print("Dimensões do conjunto de treino:", X_train.shape, y_train.shape)
print("Dimensões do conjunto de teste:", X_test.shape, y_test.shape)

# Armazenar os datasets resultantes em arquivos Parquet
def save_data(X, y, file_path):

    # Criar o diretório se não existir
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # Concatenar os recursos e o alvo
    data = pd.concat([X, y], axis=1)
    
    # Salvar os dados em formato Parquet
    data.to_parquet(file_path, index=False)

# Salvar os conjuntos de dados em arquivos Parquet
save_data(X_train, y_train, "Proj_Kobe-Bryant-Shot-Selection/data/processed/base_train.parquet")
save_data(X_test, y_test, "Proj_Kobe-Bryant-Shot-Selection/data/processed/base_test.parquet")

print("Conjuntos de dados salvos com sucesso.")

Dimensões do conjunto de treino: (16228, 6) (16228,)
Dimensões do conjunto de teste: (4057, 6) (4057,)
Conjuntos de dados salvos com sucesso.
